In [1]:
!pip install selenium


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00


In [2]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install -yq chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [783 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,356 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,048 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,906 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,091 kB]
Hit:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:13 

In [3]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [4]:
from selenium import webdriver


In [5]:
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
from bs4 import BeautifulSoup,Comment
import os

In [1]:

AUTH = 'brd-customer-hl_80709a30-zone-scraping_browser_newtsuf:w1ziplb2u1xv'
SBR_WEBDRIVER = f'https://{AUTH}@brd.superproxy.io:9515'

def append_to_json_file(file_path, new_data):
    try:
        with open(file_path, 'r+') as file:
            # Read existing data
            file_data = json.load(file)
            # Append new data
            file_data.extend(new_data)
            # Move the pointer to the beginning of the file
            file.seek(0)
            # Write the updated data
            json.dump(file_data, file, indent=4)
    except FileNotFoundError:
        # If the file does not exist, create it with the new data
        with open(file_path, 'w') as file:
            json.dump(new_data, file, indent=4)


def get_html_as_json(driver, url):
    driver.get(url)
    time.sleep(1)  # Adjust depending on page load times
    page_source = driver.page_source
    return {"url": url, "html": page_source}


def get_details_from_drive(url, page_source):
    print(url)
    # driver.get(url)
    # time.sleep(1)
    # page_source = driver.page_source
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # with the help of beautifulSoup and html parser create soup

    # Scrape the required details
    dict_job = {}
    # Find the job title
    job_title_tag = soup.find('h1', {'class': 'top-card-layout__title'})
    dict_job['job_title'] = job_title_tag.get_text(strip=True) if job_title_tag else 'Not available'
    company_name_tag=soup.find('a', {'class': 'topcard__org-name-link'})
    dict_job['company_name'] = company_name_tag.get_text(strip=True) if company_name_tag  else 'Not available'
    company_location_tag=soup.find('span', {'class': 'topcard__flavor--bullet'})
    dict_job['location'] = company_location_tag.get_text(strip=True) if company_location_tag else 'Not available'
    # print(dict_job)

    about_tag=soup.find('div', {'class': 'show-more-less-html__markup'})
    dict_job['about'] = about_tag.get_text(strip=True) if about_tag else 'Not available'

    # Scrape the workplace type, employment type, and level
    # Find all li elements with the class 'description__job-criteria-item'
    job_criteria_items = soup.find_all('li', {'class':'description__job-criteria-item'})
    # Iterate through each li element and extract the required info
    for item in job_criteria_items:
        subheader = item.find('h3', class_='description__job-criteria-subheader')
        if subheader and subheader.get_text(strip=True).lower() == 'seniority level':
            seniority_level_tag=item.find('span', class_='description__job-criteria-text description__job-criteria-text--criteria')
            dict_job['seniority_level'] = seniority_level_tag.get_text(strip=True) if seniority_level_tag else 'Not available'
        elif subheader and subheader.get_text(strip=True).lower() == 'employment type':
            empolyment_type_tag=item.find('span', class_='description__job-criteria-text description__job-criteria-text--criteria')
            dict_job['employment_type'] = empolyment_type_tag.get_text(strip=True) if empolyment_type_tag else 'Not available'



    # # Find the script tag containing the JSON data
    # json_data = None
    # script_tags = soup.find_all('script', {'type': 'application/ld+json'})
    # for script in script_tags:
    #     try:
    #         json_data = json.loads(script.string)
    #         if '@type' in json_data and json_data['@type'] == 'JobPosting':
    #             break
    #     except json.JSONDecodeError:
    #         continue  # Skip if not a valid JSON
    return {"url": url, "dict": dict_job}


def main():
    print('Connecting to Scraping Browser...')
    sbr_connection = ChromiumRemoteConnection(SBR_WEBDRIVER, 'goog', 'chrome')
    with Remote(sbr_connection, options=ChromeOptions()) as driver:
        current_job_count = 0
        page_number = 45

        while current_job_count < 1000:
            driver.get(f'https://www.linkedin.com/jobs/search/?origin=JOB_SEARCH_PAGE_JOB_FILTER?start={page_number * 25}')
            try:
        # Wait for the job listings to be present
                job_listings = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.base-card__full-link")))
                # Extract hrefs
                hrefs = [listing.get_attribute('href') for listing in job_listings]
            except Exception as e:
                print("An error occurred:", e)
                break  # Break the loop if an error occurs
            batch_content = []
            for href in hrefs:
                html_data = get_html_as_json(driver, href)
                job_details = get_details_from_drive(html_data['url'], html_data['html'])
                batch_content.append(job_details)
                current_job_count += 1
                if current_job_count >= 1000:
                    break

            # Append batch content to JSON file
            append_to_json_file('scraped_data_job_postings_20_03.json', batch_content)

            # Increment page number to move to the next page
            page_number += 1
            #print(f'Processed page {page_number}. Total jobs processed: {current_job_count}')

    print('Finished scraping 1000 job postings!')


main()



Finished scraping 1000 job postings!
